In [22]:
import xarray as xr
import dask

ds = xr.open_mfdataset(sorted_list)

# Calculate the mean and standard deviation across the time dimension
mean = ds.mean(dim='time', skipna=True)
std_dev = ds.std(dim='time', skipna=True)

# Compute z-scores (the number of standard deviations away from the mean)
z_scores = (ds - mean) / std_dev

# Identify outliers (where the absolute z-score is greater than 3)
outliers = abs(z_scores) > 3

# Replace outliers with NaN to clean the data
cleaned_data = ds.where(~outliers, other=np.nan)

# Example: Printing out or storing the actual outlier values
outlier_values = ds.where(outliers, drop=True)
print("Outlier values:")
print(outlier_values)

# Visualization: Plotting the first time slice to visualize data
# Check if the time slice or other dimensions need adjustments based on your data structure
ds.isel(time=0).plot()
plt.title('First Time Slice of Original Data')
plt.show()

cleaned_data.isel(time=0).plot()
plt.title('First Time Slice after Removing Outliers')
plt.show()

# Save cleaned data to a new NetCDF file
cleaned_data.to_netcdf('cleaned_data_file.nc')

# Close the dataset
ds.close()

/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/numpy/core/_methods.py:47: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


MemoryError: Unable to allocate 8.05 GiB for an array with shape (288, 2175, 1725) and data type float64

In [18]:
# Calculate mean, std, and z-scores using xarray's built-in methods that leverage dask for computation
mean = ds.mean(dim='time', skipna=True)
std_dev = ds.std(dim='time', skipna=True)
z_scores = (ds - mean) / std_dev

# Identify outliers
outliers = abs(z_scores) > 3

In [21]:
ds

<xarray.Dataset>
Dimensions:                  (time: 26490, projection_y_coordinate: 2175, projection_x_coordinate: 1725)
Coordinates:
  * time                     (time) datetime64[ns] 2012-06-01 ... 2012-08-31T...
  * projection_y_coordinate  (projection_y_coordinate) float32 -6.245e+05 ......
  * projection_x_coordinate  (projection_x_coordinate) float32 -4.045e+05 ......
    forecast_period          int32 0
    forecast_reference_time  (time) datetime64[ns] dask.array<chunksize=(288,), meta=np.ndarray>
Data variables:
    rainfall_rate_composite  (time, projection_y_coordinate, projection_x_coordinate) float32 dask.array<chunksize=(288, 2175, 1725), meta=np.ndarray>
    transverse_mercator      (time) int32 -2147483647 ... -2147483647
Attributes:
    field_code:                   213
    institution:                  Met Office
    nimrod_version:               2
    probability_period_of_event:  0
    source:                       Plr single site radars
    title:                        Unknown
    Conventions:                  CF-1.7

In [20]:
outliers

<xarray.Dataset>
Dimensions:                  (time: 26490, projection_y_coordinate: 2175, projection_x_coordinate: 1725)
Coordinates:
  * time                     (time) datetime64[ns] 2012-06-01 ... 2012-08-31T...
  * projection_y_coordinate  (projection_y_coordinate) float32 -6.245e+05 ......
  * projection_x_coordinate  (projection_x_coordinate) float32 -4.045e+05 ......
    forecast_period          int32 0
    forecast_reference_time  (time) datetime64[ns] dask.array<chunksize=(288,), meta=np.ndarray>
Data variables:
    rainfall_rate_composite  (time, projection_y_coordinate, projection_x_coordinate) bool dask.array<chunksize=(288, 2175, 1725), meta=np.ndarray>
    transverse_mercator      (time) bool False False False ... False False False

In [1]:
def find_all_max_locations_in_timeslices(cube):
    """
    Finds the maximum value in each timeslice and returns all its locations.
    
    Args:
        cube (iris.cube.Cube): The Iris cube to analyze.

    Returns:
        list: Each element is a tuple containing the timeslice index, maximum value, 
              and a list of coordinates where the maximum occurs.
    """
    results = []
    time_dimension = 0  # Adjust if your time dimension is not the first dimension
    
    # Iterate over each timeslice
    for i, timeslice in enumerate(cube.slices_over(time_dimension)):
        max_value = np.max(timeslice.data)
        # Find all positions where the timeslice data equals the maximum value
        positions = np.argwhere(timeslice.data == max_value)
        
        # Get the corresponding latitude and longitude for each position
        max_locations = []
        for pos in positions:
            latitude = timeslice.coord('projection_y_coordinate').points[pos[0]]
            longitude = timeslice.coord('projection_x_coordinate').points[pos[1]]
            max_locations.append((latitude, longitude))
        
        # Store the results
        results.append((i, max_value, max_locations))
    
    return results


In [2]:
import iris
import glob
import iris.plot as iplt
import iris.quickplot as qplt
import datetime as datetime
import iris.coord_categorisation as cat
import sys
import numpy as np
import pandas as pd

In [13]:
year=2012
radardir = f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{year}/'
file_list=glob.glob(radardir +"*.nc")
sorted_list = sorted(file_list)

In [4]:
for year in range(2010,2018):
    radardir = f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{year}/'
    file_list=glob.glob(radardir +"*.nc")
    sorted_list = sorted(file_list)
    day_cube = iris.load_cube(sorted_list[1])
    numbers, counts = np.unique(day_cube.data.data, return_counts=True)
    df=pd.DataFrame({'numbers':numbers, 'counts':counts})
    df = df.sort_values(by='numbers', ascending=False)
    print(df[:6])

           numbers     counts
1295  9.969210e+36  763566197
1294  7.614688e+02          1
1293  6.134375e+02          1
1292  5.923438e+02          1
1291  5.498750e+02          1
1290  3.275625e+02          1
          numbers     counts
621  9.969210e+36  773604565
620  1.023969e+03          1
619  1.746875e+02          1
618  1.619688e+02          1
617  1.185938e+02          1
616  9.575000e+01          1
           numbers     counts
4690  9.969210e+36  769132051
4689  1.023969e+03         36
4688  1.018750e+03          1
4687  1.009969e+03          1
4686  1.001156e+03          1
4685  1.000312e+03          1
           numbers     counts
1984  9.969210e+36  772067637
1983  1.023969e+03         21
1982  1.009031e+03          1
1981  1.008156e+03          1
1980  9.748125e+02          1
1979  8.835938e+02          1
           numbers     counts
1818  9.969210e+36  758441912
1817  1.023969e+03         45
1816  6.745938e+02          1
1815  4.988125e+02          1
1814  4.691562e+0

In [ ]:
# for i in range(0,len(sorted_list)):
#     print(i)
#     ### Load radar data for one day (using IRIS)
#     day_cube = iris.load_cube(sorted_list[i])
#     print(np.nanmax(day_cube.data))

### Check for repeated occurrence of value

In [7]:
for i in range(0,5):
    day_cube = iris.load_cube(sorted_list[i])
    numbers, counts = np.unique(day_cube.data.data, return_counts=True)
    df=pd.DataFrame({'numbers':numbers, 'counts':counts})
    df = df.sort_values(by='numbers', ascending=False)
    print(df[:3])

           numbers     counts
1467  9.969210e+36  768143373
1466  1.023969e+03        222
1465  1.015344e+03          1
           numbers     counts
4690  9.969210e+36  769132051
4689  1.023969e+03         36
4688  1.018750e+03          1
           numbers     counts
1719  9.969210e+36  770928656
1718  1.023969e+03          9
1717  9.740625e+02          1
           numbers     counts
2376  9.969210e+36  771696266
2375  1.023969e+03          1
2374  1.005875e+03          1
           numbers     counts
2103  9.969210e+36  779709233
2102  1.023969e+03         18
2101  9.890312e+02          1


### Check spatial/temporal occurrence of value

In [ ]:
# day_cube = iris.load_cube(sorted_list[2])

In [14]:
global_max_value = np.nanmax(day_cube.data)

In [6]:
import iris
import numpy as np

def find_specific_value_locations_in_timeslices(cube, specific_value):
    """
    Finds occurrences of a specific value in each timeslice and returns their locations.
    
    Args:
        cube (iris.cube.Cube): The Iris cube to analyze.
        specific_value (float): The specific value to search for in each timeslice.

    Returns:
        list: Each element is a tuple containing the timeslice index and a list of coordinates 
              where the specific value occurs.
    """
    results = []
    time_dimension = 0  # Adjust if your time dimension is not the first dimension
    
    # Iterate over each timeslice
    for i, timeslice in enumerate(cube.slices_over(time_dimension)):
        # Find all positions where the timeslice data equals the specific value
        positions = np.argwhere(timeslice.data == specific_value)
        
        # Get the corresponding latitude and longitude for each position
        specific_value_locations = []
        for pos in positions:
            latitude = timeslice.coord('projection_y_coordinate').points[pos[0]]
            longitude = timeslice.coord('projection_x_coordinate').points[pos[1]]
            specific_value_locations.append((latitude, longitude))
        
        # Store the results if there are any occurrences of the specific value
        if positions.size > 0:
            results.append((i, specific_value_locations))
    
    return results

# Example usage
specific_value = 1023.96875
locations_of_specific_value = find_specific_value_locations_in_timeslices(day_cube, specific_value)
for result in locations_of_specific_value:
    print(f"Timeslice Index: {result[0]}, Locations: {result[1]}")


Timeslice Index: 16, Locations: [(-26500.0, 169500.0)]
Timeslice Index: 61, Locations: [(9500.0, 209500.0)]
Timeslice Index: 87, Locations: [(-4500.0, 165500.0)]


In [7]:
day_cube = iris.load_cube(sorted_list[i])
locations_of_specific_value = find_specific_value_locations_in_timeslices(day_cube, specific_value)
for result in locations_of_specific_value:
    print(f"Timeslice Index: {result[0]}, Locations: {result[1]}")

           numbers     counts
4359  9.969210e+36  773348199
4358  1.023969e+03          3
4357  5.861875e+02          1


In [10]:
day_cube = iris.load_cube(sorted_list[1])
locations_of_specific_value = find_specific_value_locations_in_timeslices(day_cube, specific_value)
for result in locations_of_specific_value:
    print(f"Timeslice Index: {result[0]}, Locations: {result[1]}")


Timeslice Index: 16, Locations: [(-26500.0, 169500.0)]
Timeslice Index: 61, Locations: [(9500.0, 209500.0)]
Timeslice Index: 87, Locations: [(-4500.0, 165500.0)]


In [11]:
day_cube = iris.load_cube(sorted_list[2])
locations_of_specific_value = find_specific_value_locations_in_timeslices(day_cube, specific_value)
for result in locations_of_specific_value:
    print(f"Timeslice Index: {result[0]}, Locations: {result[1]}")


Timeslice Index: 138, Locations: [(337500.0, -12500.0), (337500.0, -11500.0)]
Timeslice Index: 151, Locations: [(337500.0, -37500.0), (337500.0, -36500.0), (337500.0, -35500.0), (338500.0, -36500.0), (338500.0, -35500.0), (338500.0, -34500.0), (338500.0, -33500.0), (339500.0, -24500.0), (339500.0, -23500.0), (340500.0, -24500.0), (340500.0, -23500.0), (340500.0, -22500.0), (341500.0, -13500.0), (341500.0, -12500.0), (341500.0, -11500.0), (342500.0, -13500.0), (342500.0, -12500.0), (342500.0, -11500.0)]
Timeslice Index: 227, Locations: [(336500.0, -23500.0), (336500.0, -22500.0), (337500.0, -23500.0), (337500.0, -22500.0), (337500.0, -21500.0)]


# Part 2:

In [17]:
import iris
import glob
import numpy as np
from scipy import stats

# Define the directory and create a sorted list of file paths
year = 2012
radardir = f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{year}/'
file_list = glob.glob(radardir + "*.nc")
sorted_list = sorted(file_list)

# Load the cubes
cubes = iris.load(sorted_list)

# Concatenate into a single cube
monthly_cube = cubes.concatenate_cube()


### Sum over time

### Load data and concatenate

In [18]:
# Assuming the time dimension is the first dimension
accumulated_cube = monthly_cube.collapsed('time', iris.analysis.SUM)

/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/coords.py:1982: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/coords.py:1982: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'forecast_reference_time'.
  warnings.warn(msg.format(self.name()))


### Identify outlying values

In [ ]:
# Calculate the mean and standard deviation
mean = accumulated_cube.data.mean()
std_dev = accumulated_cube.data.std()

# Compute z-scores
z_scores = (accumulated_cube.data - mean) / std_dev

# Identify outliers, e.g., where the z-score is greater than 3 or less than -3
outliers = np.abs(z_scores) > 3

### Handle outliers

In [ ]:
# Get indices where there are outliers
outlier_indices = np.where(outliers)

# Optional: Print the locations and values of outliers
for index in zip(*outlier_indices):
    print(f"Outlier at index {index} with value {accumulated_cube.data[index]} and z-score {z_scores[index]}")
